<a href="https://colab.research.google.com/github/brianramos/bots/blob/master/Entropic_Path_Heuristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import time
import statistics

class City:
    def __init__(self, x, y, name=None):
        self.x = x
        self.y = y
        self.name = name if name else str(id(self))
        self.visited = False
        self.entropy = 1.0  # Initial entropy

def calculate_global_entropy(cities, unvisited_cities):
    """Calculates global entropy based on the distribution of unvisited cities."""
    # Example using a grid-based approach:
    width = max(city.x for city in cities)
    height = max(city.y for city in cities)

    # Dynamically adjust num_cells based on width or height (example)
    num_cells = int(max(width, height) / 10) + 1  # Ensure grid covers the city space

    # Create a grid with dimensions of 'num_cells x num_cells'
    grid = [[0] * num_cells for _ in range(num_cells)]

    cell_width = width / num_cells
    cell_height = height / num_cells

    for city in unvisited_cities:
        # Ensure cell_x and cell_y are within the grid boundaries
        cell_x = int(city.x / cell_width)
        cell_y = int(city.y / cell_height)

        # Prevent cell_x and cell_y from exceeding the grid index range
        cell_x = min(cell_x, num_cells - 1)
        cell_y = min(cell_y, num_cells - 1)

        grid[cell_x][cell_y] += 1

    # Calculate entropy based on cell counts
    total_cities = len(unvisited_cities)
    entropy = 0
    for row in grid:
        for count in row:
            if count > 0:
                probability = count / total_cities
                entropy -= probability * math.log(probability, 2)

    return entropy

def euclidean_distance(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)

def generate_tsp_problem(num_cities, width, height):
    cities = []
    for i in range(num_cities):
        cities.append(City(random.randint(0, width), random.randint(0, height), name=f"City {i}"))
    return cities

def path_length(path):
    length = 0
    for i in range(len(path) - 1):
        length += euclidean_distance(path[i], path[i+1])
    length += euclidean_distance(path[-1], path[0])  # Close the loop
    return length

def nearest_neighbor(cities):
    num_cities = len(cities)
    path = []
    current_city = random.choice(cities)
    current_city.visited = True
    path.append(current_city)

    for _ in range(num_cities - 1):
        min_distance = float('inf')
        next_city = None
        for city in cities:
            if not city.visited:
                distance = euclidean_distance(current_city, city)
                if distance < min_distance:
                    min_distance = distance
                    next_city = city

        if next_city:
            next_city.visited = True
            path.append(next_city)
            current_city = next_city
    return path

def two_opt(cities):
  num_cities = len(cities)
  path = cities.copy()
  improved = True
  while improved:
    improved = False
    for i in range(1, num_cities - 2):
        for j in range(i + 1, num_cities):
            if j - i == 1: continue  # changes nothing, skip
            new_path = path[:i] + path[i:j][::-1] + path[j:]
            if path_length(new_path) < path_length(path):
                path = new_path
                improved = True
                break  # improvement found, restart inner loop
        if improved:
            break  # improvement found, restart outer loop
  return path


def calculate_density(city, cities, radius):
    """Calculates the density of cities within a given radius around a city."""
    count = 0
    for other_city in cities:
        if other_city != city and euclidean_distance(city, other_city) <= radius:
            count += 1
    return count

def adaptive_contract_bubble(city, stretch, bubble_strength=0.5, density_radius=20, cities=None):  # Added cities argument
    """Contracts the bubble adaptively based on city density."""

    density = calculate_density(city, cities, density_radius)

    # Adjust bubble_strength based on density (example)
    adjusted_bubble_strength = bubble_strength * (1 - density / len(cities))

    city.entropy = max(0, city.entropy - (0.1 + 0.05 * stretch) * adjusted_bubble_strength)

def entropic_distance(city1, city2, stretch, initial_distances, global_entropy):
    """Calculates entropic distance, incorporating global entropy."""
    initial_distance = initial_distances[(city1, city2)]
    entropy_factor = 1 / (city1.entropy * city2.entropy + 1e-10)
    stretch_factor = 1 + 0.1 * stretch
    # Global entropy influence:
    global_entropy_weight = 1 + (global_entropy * 0.2)  # Example weighting
    return initial_distance * entropy_factor * stretch_factor / global_entropy_weight


def traveling_salesperson_entropic_adaptive(cities, bubble_strength=0.5, density_radius=20):
    num_cities = len(cities)
    path = []
    initial_distances = {}
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = euclidean_distance(cities[i], cities[j])
            initial_distances[(cities[i], cities[j])] = distance
            initial_distances[(cities[j], cities[i])] = distance

    current_city = random.choice(cities)
    current_city.visited = True
    path.append(current_city)
    total_stretch = 0
    unvisited_cities = cities[:]  # Copy of cities
    for _ in range(num_cities - 1):
        global_entropy = calculate_global_entropy(cities, unvisited_cities)
        min_distance = float('inf')
        next_city = None
        for city in cities:
            if not city.visited:
                distance = entropic_distance(current_city, city, total_stretch, initial_distances, global_entropy)
                if distance < min_distance:
                    min_distance = distance
                    next_city = city
        if next_city:
            total_stretch += euclidean_distance(current_city, next_city)
            adaptive_contract_bubble(current_city, total_stretch, bubble_strength, density_radius, cities)
            adaptive_contract_bubble(next_city, total_stretch, bubble_strength, density_radius, cities)
            next_city.visited = True
            path.append(next_city)
            current_city = next_city
            unvisited_cities.remove(next_city)
    return path, path_length(path)

def reset_cities(cities):
    for city in cities:
        city.visited = False
        city.entropy = 1.0

def tune_entropic_path(cities, iterations, bubble_strengths):
    """Tunes the Entropic Path method parameters.  Uses the Bard-Entropy heuristic."""
    # Core concept: Using entropy as a heuristic to guide city selection (Concept by Bard).
    best_path = None
    min_length = float('inf')
    best_params = None

    for _ in range(iterations):
        reset_cities(cities) # Reset here
        bubble_strength = random.choice(bubble_strengths)
        path, length = traveling_salesperson_entropic_adaptive(cities.copy(), bubble_strength=bubble_strength)
        if length < min_length:
            min_length = length
            best_path = path
            best_params = (bubble_strength)

    return best_path, min_length, best_params

def gather_statistics(problem_size, num_iterations):
    # Initialize lists to store results
    entropic_path_lengths = []
    entropic_runtimes = []
    nn_path_lengths = []
    nn_runtimes = []
    two_opt_path_lengths = []
    two_opt_runtimes = []

    best_solution_counts = {
        "entropic": 0,
        "nearest_neighbor": 0,
        "two_opt": 0,
    }

    for _ in range(num_iterations):
        cities = generate_tsp_problem(problem_size, 100, 100)

        # Entropic Path
        start_time = time.time()
        best_path, min_length, _ = tune_entropic_path(cities, 9, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
        end_time = time.time()
        entropic_path_lengths.append(min_length)
        entropic_runtimes.append(end_time - start_time)

        # Nearest Neighbor
        start_time = time.time()
        reset_cities(cities)
        nn_path = nearest_neighbor(cities.copy())
        end_time = time.time()
        nn_path_lengths.append(path_length(nn_path))
        nn_runtimes.append(end_time - start_time)

        # 2-opt
        start_time = time.time()
        reset_cities(cities)
        two_opt_path = two_opt(cities.copy())
        end_time = time.time()
        two_opt_path_lengths.append(path_length(two_opt_path))
        two_opt_runtimes.append(end_time - start_time)

        # Determine the best solution for this iteration
        min_length_overall = min(min_length, path_length(nn_path), path_length(two_opt_path))
        if math.isclose(min_length, min_length_overall):
          best_solution_counts["entropic"] +=1
        if math.isclose(path_length(nn_path), min_length_overall):
          best_solution_counts["nearest_neighbor"] +=1
        if math.isclose(path_length(two_opt_path), min_length_overall):
          best_solution_counts["two_opt"] += 1

    # Calculate and print statistics
    print(f"Problem Size: {problem_size}, Iterations: {num_iterations}")

    print("Entropic Path:")
    print(f"  Mean Path Length: {statistics.mean(entropic_path_lengths):.2f}")
    print(f"  Mean Runtime: {statistics.mean(entropic_runtimes):.4f} seconds")

    print("Nearest Neighbor:")
    print(f"  Mean Path Length: {statistics.mean(nn_path_lengths):.2f}")
    print(f"  Mean Runtime: {statistics.mean(nn_runtimes):.4f} seconds")

    print("2-opt:")
    print(f"  Mean Path Length: {statistics.mean(two_opt_path_lengths):.2f}")
    print(f"  Mean Runtime: {statistics.mean(two_opt_runtimes):.4f} seconds")

    print("Best Solution Counts:")
    for method, count in best_solution_counts.items():
      print(f" {method}: {count}")

# Example Usage
gather_statistics(30, 100)